In [6]:
from data_loader import load_data

In [33]:
import numpy as np
import random
import math

def swap_move(solution, n_depots, demands, capacities):
    neighbor = solution.copy()
    n_customers = len(solution)
    # Rastgele iki müşteri seç
    i, j = random.sample(range(n_customers), 2)
    
    # İki müşteri arasındaki depo atamalarını değiştir
    if capacities[neighbor[j]] >= demands[i] and capacities[neighbor[i]] >= demands[j]:
        neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
    return neighbor

def simulated_annealing(current_solution, n_depots, demands, capacities, setup_costs, costs, max_iterations=500, initial_temperature=100, cooling_rate=0.99):
    current_cost = calculate_total_cost(current_solution, setup_costs, costs, demands, capacities)
    best_solution = current_solution.copy()
    best_cost = current_cost
    temperature = initial_temperature
    
    for iteration in range(max_iterations):
        # Yeni bir komşu çözüm oluştur
        neighbor = swap_move(current_solution, n_depots, demands, capacities)
        neighbor_cost = calculate_total_cost(neighbor, setup_costs, costs, demands, capacities)
        
        # Simulated Annealing kabul kriteri
        if neighbor_cost < current_cost:
            current_solution = neighbor
            current_cost = neighbor_cost
        else:
            # Kötü çözüm de olsa, belirli bir olasılıkla kabul et
            probability = math.exp((current_cost - neighbor_cost) / temperature)
            if random.random() < probability:
                current_solution = neighbor
                current_cost = neighbor_cost
        
        # Eğer daha iyi bir çözüm bulunduysa, kaydet
        if current_cost < best_cost:
            best_solution = current_solution.copy()
            best_cost = current_cost
        
        # Sıcaklık düşürülür
        temperature *= cooling_rate

    return best_solution, best_cost

def tabu_search_with_sa(n_depots, n_customers, capacities, setup_costs, demands, costs, max_iterations=500, tabu_tenure=20, patience=50):
    # İlk çözüm rastgele alınıyor
    current_solution = random_initial_solution(n_depots, n_customers, capacities, demands)
    best_solution = current_solution.copy()
    best_cost = calculate_total_cost(best_solution, setup_costs, costs, demands, capacities)
    tabu_list = []
    no_improve = 0
    
    for it in range(max_iterations):
        # Simulated Annealing ile local search yapalım
        current_solution, current_cost = simulated_annealing(current_solution, n_depots, demands, capacities, setup_costs, costs)
        
        # Aspiration Criteria
        move = tuple(current_solution)
        if move in tabu_list:
            if current_cost < best_cost:
                print(f"✅ Aspiration accepted a tabu move at iteration {it}")
                best_solution = current_solution.copy()
                best_cost = current_cost
                no_improve = 0
                continue
            else:
                continue
        
        # Yeni çözüm bulunduğunda, tabu listesine ekle
        if current_cost < best_cost:
            best_cost = current_cost
            best_solution = current_solution.copy()
            no_improve = 0
        else:
            no_improve += 1

        # Tabu listesine ekle ve maksimum uzunluğu kontrol et
        tabu_list.append(move)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)

        if no_improve >= patience:
            print(f"🛑 Early stopping at iteration {it}")
            break

    return best_solution, best_cost


In [34]:
filename = 'Dataset/wl_25'
n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs = load_data(filename)

solution, cost = tabu_search_with_sa(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs)
print("Total cost:", cost)
print("Solution:", solution)


Total cost: 890314.975
Solution: [7, 13, 0, 17, 1, 0, 1, 2, 16, 1, 3, 4, 5, 17, 6, 7, 3, 5, 3, 13, 2, 9, 3, 0, 11, 5, 12, 10, 13, 14, 15, 13, 15, 16, 19, 11, 17, 18, 21, 18, 19, 20, 7, 9, 12, 21, 20, 15, 24, 19]


In [35]:
filename = 'Dataset/wl_50'
n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs = load_data(filename)

solution, cost = tabu_search_with_sa(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs)
print("Total cost:", cost)
print("Solution:", solution)


Total cost: 1103167.3875
Solution: [0, 15, 25, 3, 15, 25, 36, 3, 42, 9, 10, 27, 37, 39, 34, 15, 24, 17, 48, 19, 39, 32, 10, 29, 24, 25, 47, 27, 28, 29, 30, 31, 30, 33, 34, 35, 36, 37, 42, 39, 27, 41, 42, 30, 44, 46, 42, 44, 48, 49]


In [36]:
filename = 'Dataset/wl_200'
n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs = load_data(filename)

solution, cost = tabu_search_with_sa(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs)
print("Total cost:", cost)
print("Solution:", solution)


Total cost: 46175.70395
Solution: [193, 124, 46, 154, 87, 91, 78, 54, 170, 88, 109, 189, 125, 25, 114, 21, 158, 41, 29, 100, 156, 99, 160, 51, 128, 181, 86, 180, 105, 57, 34, 118, 18, 6, 95, 182, 102, 29, 33, 142, 15, 25, 73, 14, 35, 158, 96, 30, 145, 152, 184, 81, 31, 113, 108, 48, 16, 106, 82, 133, 15, 49, 2, 185, 58, 20, 88, 56, 100, 193, 94, 48, 130, 58, 152, 71, 165, 116, 199, 10, 100, 3, 170, 34, 53, 65, 34, 116, 45, 15, 190, 13, 121, 56, 132, 177, 4, 178, 121, 53, 38, 160, 97, 112, 93, 4, 75, 107, 187, 178, 183, 44, 198, 132, 83, 168, 41, 25, 77, 7, 177, 188, 163, 77, 92, 138, 30, 65, 135, 170, 179, 181, 162, 108, 112, 190, 141, 113, 173, 40, 199, 91, 105, 151, 143, 65, 82, 155, 182, 192, 179, 4, 176, 39, 68, 133, 176, 59, 94, 189, 12, 53, 93, 62, 112, 134, 187, 106, 149, 145, 196, 75, 185, 26, 80, 71, 38, 99, 96, 21, 158, 79, 5, 179, 177, 44, 142, 80, 3, 185, 174, 78, 42, 40, 146, 38, 141, 152, 14, 130]


In [37]:
filename = 'Dataset/wl_300'
n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs = load_data(filename)

solution, cost = tabu_search_with_sa(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs)
print("Total cost:", cost)
print("Solution:", solution)



Total cost: 108084.3688
Solution: [281, 182, 11, 252, 50, 110, 48, 8, 214, 82, 28, 38, 178, 263, 19, 113, 179, 0, 12, 278, 259, 77, 149, 122, 101, 120, 185, 120, 134, 127, 262, 30, 275, 285, 263, 287, 266, 223, 20, 105, 239, 111, 15, 146, 281, 215, 109, 48, 158, 68, 280, 85, 281, 259, 81, 166, 10, 4, 79, 294, 224, 114, 30, 198, 285, 152, 63, 86, 273, 172, 254, 103, 256, 244, 25, 171, 6, 179, 298, 122, 31, 197, 169, 34, 180, 120, 168, 19, 291, 110, 127, 93, 171, 173, 140, 124, 245, 54, 244, 299, 273, 140, 291, 210, 39, 4, 271, 239, 235, 148, 20, 182, 153, 29, 45, 212, 85, 77, 102, 14, 18, 282, 204, 29, 179, 233, 33, 277, 246, 30, 160, 24, 258, 221, 151, 271, 109, 29, 269, 146, 122, 240, 134, 277, 118, 290, 218, 22, 121, 214, 295, 119, 74, 194, 116, 77, 83, 113, 111, 295, 92, 281, 118, 286, 27, 183, 265, 171, 206, 52, 172, 169, 287, 163, 187, 110, 108, 152, 145, 203, 29, 292, 201, 2, 178, 201, 206, 284, 52, 101, 137, 232, 66, 211, 233, 101, 214, 268, 101, 183, 242, 137, 220, 7, 68, 182, 

In [38]:
filename = 'Dataset/wl_500'
n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs = load_data(filename)

solution, cost = tabu_search_with_sa(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs)
print("Total cost:", cost)
print("Solution:", solution)


Total cost: 104267.07342
Solution: [193, 348, 71, 375, 102, 289, 131, 136, 102, 168, 440, 362, 455, 31, 322, 379, 265, 257, 439, 300, 329, 163, 278, 277, 330, 100, 388, 239, 37, 314, 318, 255, 462, 266, 242, 389, 12, 469, 442, 250, 455, 201, 189, 414, 292, 426, 437, 233, 311, 136, 397, 227, 386, 288, 413, 115, 56, 458, 483, 469, 444, 400, 216, 327, 98, 428, 282, 371, 104, 407, 420, 451, 208, 238, 403, 69, 423, 294, 140, 52, 439, 95, 187, 437, 442, 20, 40, 312, 458, 444, 482, 70, 92, 262, 81, 242, 83, 97, 272, 121, 432, 83, 283, 117, 300, 419, 80, 442, 430, 490, 130, 89, 398, 124, 236, 74, 45, 202, 299, 463, 402, 135, 328, 7, 97, 64, 182, 103, 150, 81, 237, 464, 156, 13, 24, 416, 197, 237, 64, 280, 13, 366, 42, 246, 94, 369, 136, 308, 491, 209, 446, 333, 270, 98, 319, 184, 190, 205, 69, 267, 364, 157, 48, 441, 194, 49, 451, 189, 460, 16, 320, 478, 46, 397, 103, 184, 485, 111, 244, 102, 68, 93, 273, 473, 449, 114, 134, 396, 194, 495, 42, 156, 29, 41, 1, 292, 296, 262, 89, 406, 419, 217, 